# Init

In [1]:
roi_file = "./roi/land.shp"  # Replace with your ROI file path
output_dir = "./data"  # Replace with your output directory path

# 定义日期范围（替换为你需要的日期）
start_date = "2023-01-01"  # 开始日期
end_date = "2023-01-15"    # 结束日期
datetime_range = f"{start_date}/{end_date}"

import geopandas as gpd
roi = gpd.read_file(roi_file).to_crs('epsg:4326')  # Replace with your ROI file path

roi_geometry = roi.unary_union
minx, miny, maxx, maxy = roi.total_bounds
bbox = [minx, miny, maxx, maxy]

import os 
output_dir = os.path.abspath(output_dir)
os.makedirs(output_dir, exist_ok=True)

C:\Users\19735\AppData\Local\Temp\ipykernel_498472\2743640760.py:12: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  roi_geometry = roi.unary_union


# Eodag
参考：https://github.com/CS-SI/eodag



For downloading you will need to fill your credentials for the desired providers in [your eodag user configuration file](https://eodag.readthedocs.io/en/latest/getting_started_guide/configure.html). The file will automatically be created with empty values on the first run.

In [ ]:
# %conda install -c conda-forge eodag

In [ ]:
from eodag import EODataAccessGateway, setup_logging

setup_logging(1) # 0: nothing, 1: only progress bars, 2: INFO, 3: DEBUG

dag = EODataAccessGateway()
dag.set_preferred_provider("cop_dataspace")  # 设置首选提供商为 cop_dataspace
# dag.update_providers_config(f"""
#     cop_dataspace:
#         auth:
#             credentials:
#                 username: # Replace with your username
#                 password: # Replace with your password
#         download:
#             extract: false   # 是否解压下载的产品，默认为 true
#             delete_archive: false # 是否删除下载的归档文件，默认为 true
#             output_dir: {output_dir}  # maybe output_prefix
# """)

# 查找产品名称
# dag.guess_product_type(keywords="S1")
# dag.available_providers('S1_SAR_GRD')  check available providers for a product type

products = dag.search(
  productType="S1_SAR_GRD",
  geom=roi_geometry,
  start=start_date,
  end=end_date,
  # cloudCover=51,
)
print(f"Found {len(products)} products")


Found 8 products


In [ ]:
import folium

# 获取多边形的中心点坐标
center = [roi_geometry.centroid.y, roi_geometry.centroid.x]

emap = folium.Map(location=center, zoom_start=4)
layer = folium.features.GeoJson(
    data=products.as_geojson_object(),
    tooltip=folium.GeoJsonTooltip(fields=['title'])
).add_to(emap)
emap

In [3]:
product_path = products[0].download()

product_path

0.00B [00:00, ?B/s]

'F:\\BaiduNetdiskDownload\\Python\\DockerS1Pre\\data\\S1A_EW_GRDM_1SDH_20230101T080118_20230101T080222_046587_059549_9BF0'

In [ ]:
paths = dag.download_all(
    products,
    output_dir=output_dir,
)
paths

# Geoai
参考：https://opengeoai.org/examples/planetary_computer/

In [ ]:
# %pip install geoai-py --ignore-installed blinker

In [55]:
import geoai

In [56]:
collections = geoai.pc_collection_list()
collections

Retrieved 126 collections from Planetary Computer


,id,title,description
42,3dep-lidar-classification,USGS 3DEP Lidar Classification,This collection is derived from the [USGS 3DEP...
28,3dep-lidar-copc,USGS 3DEP Lidar Point Cloud,This collection contains source data from the ...
3,3dep-lidar-dsm,USGS 3DEP Lidar Digital Surface Model,This collection is derived from the [USGS 3DEP...
43,3dep-lidar-dtm,USGS 3DEP Lidar Digital Terrain Model,This collection is derived from the [USGS 3DEP...
41,3dep-lidar-dtm-native,USGS 3DEP Lidar Digital Terrain Model (Native),This collection is derived from the [USGS 3DEP...
...,...,...,...
15,terraclimate,TerraClimate,[TerraClimate](http://www.climatologylab.org/t...
55,us-census,US Census,The [2020 Census](https://www.census.gov/progr...
84,usda-cdl,USDA Cropland Data Layers (CDLs),The Cropland Data Layer (CDL) is a product of ...
89,usgs-lcmap-conus-v13,USGS LCMAP CONUS Collection 1.3,"The [Land Change Monitoring, Assessment, and P..."


In [ ]:
items = geoai.pc_stac_search(
    collection="sentinel-1-grd",
    bbox=bbox,  
    time_range=datetime_range,
    max_items=10,
)

items

Found 8 items matching search criteria


[<Item id=S1A_EW_GRDM_1SDH_20230113T080117_20230113T080221_046762_059B29>,
 <Item id=S1A_EW_GRDM_1SDH_20230111T081731_20230111T081835_046733_059A35>,
 <Item id=S1A_IW_GRDH_1SSH_20230108T110719_20230108T110744_046691_0598C4>,
 <Item id=S1A_EW_GRDM_1SDH_20230108T075312_20230108T075416_046689_0598B0>,
 <Item id=S1A_EW_GRDM_1SDH_20230107T071233_20230107T071337_046674_059825>,
 <Item id=S1A_EW_GRDM_1SDH_20230106T080926_20230106T081030_046660_0597B4>,
 <Item id=S1A_EW_GRDM_1SDH_20230104T082540_20230104T082644_046631_0596C9>,
 <Item id=S1A_EW_GRDM_1SDH_20230101T080118_20230101T080222_046587_059549>]

In [ ]:
# 打印item的文件信息
geoai.pc_item_asset_list(items[0])
# 可视化
# geoai.view_pc_item(item=items[0],  basemap="SATELLITE")

['hh',
 'hv',
 'thumbnail',
 'safe-manifest',
 'schema-noise-hh',
 'schema-noise-hv',
 'schema-product-hh',
 'schema-product-hv',
 'schema-calibration-hh',
 'schema-calibration-hv',
 'tilejson',
 'rendered_preview']

In [ ]:
# 可选 assets = ["data", "rendered_preview"]
geoai.pc_stac_download(items[0], output_dir=output_dir)

# Other
参考：https://github.com/Rithvij/Copernicus-STAC-API---Sentinel-1-Workflow

In [ ]:
import os
import numpy as np
import xarray as xr
from pystac_client import Client
from shapely.geometry import mapping, shape

# STAC API client setup
client = Client.open("https://catalogue.dataspace.copernicus.eu/stac") 
collection = "SENTINEL-1"

# 使用几何图形（而不仅仅是边界框）进行更精确的搜索
search = client.search(
    collections=[collection],
    bbox=bbox,  # 使用边界框
    datetime=datetime_range,  # 日期范围
    limit=200  # 增加结果限制
)

# 获取所有匹配项目
items = list(search.get_items())

# 筛选符合条件的数据（操作模式为 'EW' 且处理级别为 'LEVEL1'）
matching_items = []
for item in items:
    # 检查操作模式
    if item.properties.get('operationalMode') == 'EW':
        # 检查处理级别
        processing_level = item.properties.get('processingLevel', '')
        # print(item.properties)
        # 只选择处理级别为LEVEL1 的GRDM产品
        if processing_level == 'LEVEL1':
            # 检查几何相交（精确过滤）
            item_geometry = shape(item.geometry)
            # 计算交集面积与ROI面积的比例
            intersection_area = roi_geometry.intersection(item_geometry).area
            roi_area = roi_geometry.area
            intersection_ratio = intersection_area / roi_area
            if roi_geometry.intersects(item_geometry):
                matching_items.append(item)

print(f"找到 {len(matching_items)} 个符合条件的项目。")

找到 14 个符合条件的项目。


In [ ]:
import requests
def get_keycloak(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Keycloak token creation failed. Reponse from the server was: {r.json()}"
        )
    return r.json()["access_token"]

# 填写用户名和密码
keycloak_token = get_keycloak('xxx','xxx')
keycloak_token


In [ ]:
import os
import requests

# Function to download the item
def download_item(item):
    # keycloak_token = get_keycloak('xxx','xxx')
    
    # Access the 'PRODUCT' asset to get the download link
    product_asset = item.assets.get('PRODUCT')
    
    if product_asset:
        # Extract the product ID from the asset URL
        product_id = product_asset.href.split('(')[1].split(')')[0]  # Extract the product ID
        
        # Construct the download URL for the ZIP file
        download_url = f'https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value'

        print(f"Downloading from: {download_url}")
        
        # Optional: Use headers for authorization if needed
        headers = {
            'Authorization': f"Bearer {keycloak_token}"  # Replace with your actual token if needed
        }
        
        # Download the file
        response = requests.get(download_url, headers=headers)
        
        if response.status_code == 200:
            # Save the file as a ZIP
            file_name = os.path.join(output_dir, f"{item.id}.zip")  # Change the extension to .zip
            with open(file_name, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {file_name}")
        else:
            print(f"Failed to download {item.id} with status code: {response.status_code}")
            print(f"Error Message: {response.text if response.text else 'No additional error message'}")
    else:
        print(f"No PRODUCT asset found for {item.id}")

#Iterate through filtered items and download each
for item in matching_items:
    download_item(item)
    break


Downloaded: ./data\S1A_EW_GRDM_1SDH_20230113T080117_20230113T080221_046762_059B29_FC3A_COG.SAFE.zip
